In [ ]:
import pathlib
import shutil
from math import ceil, sqrt
from pathlib import Path
from typing import Any, Callable, List, Optional, Sequence, Tuple

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import pyrootutils
import torch
import torchvision.transforms as T
import torchvision.transforms.functional as F
from skimage.metrics import structural_similarity
from sklearn.model_selection import GroupShuffleSplit
from torch.utils.data import ConcatDataset, Dataset
from torchmetrics import Metric
from torchvision.io import read_image
from tqdm import tqdm

root = pyrootutils.setup_root(search_from=".", indicator=".project-root", pythonpath=True)

from src.data.components.dataset import FetalPlanesDataset, USVideosDataset
from src.data.utils.utils import show_numpy_images, show_pytorch_images
from src.models.fetal_module import FetalLitModule

root

In [ ]:
torch.randint(0, 3, [])

In [ ]:
torch.randint(0, 3, (1,))

In [ ]:
a = [1, 2, 3]
i = torch.randint(0, 3, (1,))

a[i]

In [ ]:
x = torch.tensor([1], dtype=torch.float32)
print(x)
print(torch.mean(torch.cat([x])))
print(torch.mean(torch.cat([x, x, x])))

In [ ]:
x = torch.tensor([[1, 2, 3]], dtype=torch.float32)
print(x)
print(torch.mean(torch.cat([x]), dim=0))
print(torch.mean(torch.cat([x, x, x]), dim=0))

In [ ]:
torch.exp(torch.tensor(float("-inf")))

In [ ]:
loss = torch.nn.CrossEntropyLoss()

x = torch.tensor([[0.0, 0.0, 0.0]])
y = torch.tensor([1])
loss(x, y)

In [ ]:
loss = torch.nn.CrossEntropyLoss()

x = torch.tensor([[float("-inf"), float("-inf"), 0.0, 0.0, 0.0]])
y = torch.tensor([3])
loss(x, y)

In [ ]:
softmax = torch.nn.Softmax(dim=1)

x = torch.tensor([[0.0, 0.0, 0.0]])
softmax(x)

In [ ]:
softmax = torch.nn.Softmax(dim=1)

x = torch.tensor([[float("-inf"), float("-inf"), 0.0, 0.0, 0.0]])
softmax(x)

In [ ]:
x = torch.tensor([[0.0, 0.0, 0.0, 0.0, 0.0]])
mask = torch.tensor([1, 1, 0, 0, 0])
torch.masked_fill(x, mask, value=float("-inf"))

In [ ]:
masks = torch.nn.functional.one_hot(torch.arange(0, 3), num_classes=5)
print(len(masks))
masks

In [ ]:
x = torch.tensor([[0.0, 0.0, 0.0, 0.0, 0.0]])
mask = torch.tensor([1, 1, 0, 0, 0], dtype=torch.bool)
x.masked_fill(mask, value=float("-inf"))

In [ ]:
for i, c in enumerate(torch.tensor([1, 1, 0, 0, 0], dtype=torch.bool)):
    if c:
        print(i)